# Full Scenario Identification Notebook

This notebook shows the scenario identification experiments we used.
Note that this notebook uses data that we are not able to make fully public at this time. 
To run this notebook, it will be necessary to run the proper simulations and compute the appropriate aggregate measures for each cascade.
We include this notebook in order to further reproducibility.


In [ ]:
import pandas as pd
import re
from glob import glob
import os

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import KFold, StratifiedKFold, GridSearchCV, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline

from sklearn.base import clone
from copy import deepcopy

import numpy as np

In [ ]:
bin_size = 10 # size of bins to consider

# expressions to identify column types
num_expr = re.compile("infections_(\d+)_to_(\d+)")
out_degree_expr = re.compile("^out_degree_(\d+)_to_(\d+)")
bdry_expr = re.compile("boundary_out_degree_(\d+)_to_(\d+)")
path_ct_expr = re.compile("cnt_*")
avg_ct_expr = re.compile("avg_*")

In [ ]:
# Identifies which columns correspond to which bucket sizes
# We only use one bucket size - experiments found little difference with increased granularity
def is_bucket(str_val, expr, size):
    matches = expr.match(str_val)
    if not matches:
        return False
    
    lb = int(matches[1])
    ub = int(matches[2])
    
    return ub - lb == size

In [ ]:
def train_eval_model(data, bin_size, estimator, path_expr, col_set=None, n_iter=10):
    
    """
    do 10 train/evaluate splits using the specified data, and set of
    columns
    
    data - the aggregate data where each row is a cascade
    bin_size - specifies the subset of binned columns to use (e.g. 5, 10)
    estimator - an instantiated SkLearn classifier 
    path_expr - a regex indicating whether to use avg or count for labeled paths
    col_set - a tuple indicating which sets of columns to use
    
    returns - a list of 10 (accuracy, trained_estimator) tuples
    """
    
    inf_cols = data.columns[data.columns.map(lambda x: is_bucket(x, num_expr, bin_size))]
    path_cols = data.filter(regex=path_expr).columns
    
    # get all other columns
    out_degree_expr = re.compile("^out_degree_(\d+)_to_(\d+)")
    path_len_cols = data.filter(regex="^path_len_*").columns
    out_degree_cols = data.columns[data.columns.map(lambda x: is_bucket(x, out_degree_expr, 2))]

    feature_data = data[inf_cols.union(path_cols).union(path_len_cols).union(out_degree_cols)].copy()

    norm_cols = inf_cols.union(out_degree_cols).union(path_len_cols)

    feature_data[norm_cols] = feature_data[norm_cols].divide(data[inf_cols].sum(axis=1).values, axis=0)
    
    if col_set:
        cols = pd.Index([])
        if "inf_cols" in col_set:
            cols = cols.union(inf_cols)
        if "labeled_path_1" in col_set:
            labeled_path_1_ind = ~data.filter(regex=path_expr).columns.str.contains("-")
            cols = cols.union(data.filter(regex=path_expr).columns[labeled_path_1_ind])
        if "labeled_path_2" in col_set:
            labeled_path_2_ind = data.filter(regex=path_expr).columns.str.contains("-")
            cols = cols.union(data.filter(regex=path_expr).columns[labeled_path_2_ind])
        if "out_degree" in col_set:
            cols = cols.union(out_degree_cols)
        if "path_len" in col_set:
            cols = cols.union(path_len_cols)
        feature_data = feature_data[cols]

    runs = []
    
    for i in range(n_iter):
        
        X_train, X_test, y_train, y_test = train_test_split(feature_data, 
                                                        data["label_scenario"], shuffle=True, 
                                                        stratify=data["label_scenario"], test_size=0.25)
        estimator.fit(X_train, y_train)
        acc = estimator.score(X_test, y_test)
        
        est_arch = deepcopy(estimator)
      
        runs.append((acc, est_arch))
        #print(f"Iteration {i}, {acc}")

    return runs

In [ ]:
# models

col_combos = [
    ("inf_cols", "path_len", "out_degree", "labeled_path_1", "labeled_path_2"), # Epicurve + structure
    ("inf_cols",) # Epicurve only
]

# whether to use count or average, determined via exploratory testing
lr_path_expr = "cnt_en_*" 
rf_path_expr = "avg_en_*"
svm_path_expr = "avg_s_en_*"


lr = make_pipeline(LogisticRegression(max_iter=10000))
gr_lr = GridSearchCV(lr, param_grid={
            "logisticregression__C" : np.arange(0.1, 3.0, 0.05)
        }, n_jobs=4, cv=5)

cv_rf = GridSearchCV(RandomForestClassifier(), param_grid={
    "max_features" : ["sqrt", "log2"] + list(range(1,40)),
}, n_jobs=4, cv=5)

cv_svm = GridSearchCV(make_pipeline(StandardScaler(), SVC(kernel="linear", probability=True)), param_grid={
    "svc__C" : [1e-2, 1e-1, 1.0, 1e1],    
}, n_jobs=4, cv=5)


## Coverage Experiments

In [ ]:
# Load data
directory = "/project/biocomplexity/cascade_analytics/2023-KDD/aggregates/exp4.coverage_l1/"
file_expr = re.compile("cells-1-3-5-7_coverage-(\d\.\d)_seed-(\d).csv")

files = glob(directory+"*.csv")

dfs = []

for file in files:
    meta = file_expr.match(os.path.basename(file))
    coverage = float(meta[1])
    seed = int(meta[2])
    
    df = pd.read_csv(file)
    df["coverage"] = coverage
    df["seed"] = seed
    
    dfs.append(df)
    
cov_data = pd.concat(dfs)
cov_data.set_index(["coverage", "seed"], inplace=True)
cov_data.fillna(0.0, inplace=True) # missing appears to be because of incompatible columns due to binning
cov_data.sort_index(inplace=True)

In [ ]:
# Logistic Regression results

lr_results = pd.DataFrame(columns = ["struct","epicurve"],
                         index=pd.MultiIndex.from_product([
                             [0.6,0.7,0.8,0.9],
                             [1,2,3,4,5],
                             [0,1,2,3,4,5,6,7,8,9]
                         ]))

for coverage in cov_data.index.get_level_values(0).unique():
    for seed in cov_data.index.get_level_values(1).unique():
        data = cov_data.loc[(coverage, seed),:].reset_index().drop(["coverage", "seed"], axis=1)
        full_res = train_eval_model(data, 10, gr_lr, re.compile(lr_path_expr), col_set=col_combos[0])
        epi_only = train_eval_model(data, 10, gr_lr, re.compile(lr_path_expr), col_set=col_combos[1])
        
        lr_results.loc[(coverage, seed, slice(None)), "struct"] = [r[0] for r in full_res]
        lr_results.loc[(coverage, seed, slice(None)), "epicurve"] = [r[0] for r in epi_only]

In [ ]:
# Random Forest results

rf_results = pd.DataFrame(columns = ["struct","epicurve"],
                         index=pd.MultiIndex.from_product([
                             [0.6,0.7,0.8,0.9],
                             [1,2,3,4,5],
                             [0,1,2,3,4,5,6,7,8,9]
                         ]))

for coverage in cov_data.index.get_level_values(0).unique():
    for seed in cov_data.index.get_level_values(1).unique():
        data = cov_data.loc[(coverage, seed),:].reset_index().drop(["coverage", "seed"], axis=1)
        full_res = train_eval_model(data, 10, cv_rf, re.compile(rf_path_expr), col_set=col_combos[0])
        epi_only = train_eval_model(data, 10, cv_rf, re.compile(rf_path_expr), col_set=col_combos[1])
        
        rf_results.loc[(coverage, seed, slice(None)), "struct"] = [r[0] for r in full_res]
        rf_results.loc[(coverage, seed, slice(None)), "epicurve"] = [r[0] for r in epi_only]

In [ ]:
# SVM results

svm_results = pd.DataFrame(columns = ["struct","epicurve"],
                         index=pd.MultiIndex.from_product([
                             [0.6,0.7,0.8,0.9],
                             [1,2,3,4,5],
                             [0,1,2,3,4,5,6,7,8,9]
                         ]))

for coverage in cov_data.index.get_level_values(0).unique():
    for seed in cov_data.index.get_level_values(1).unique():
        data = cov_data.loc[(coverage, seed),:].reset_index().drop(["coverage", "seed"], axis=1)
        full_res = train_eval_model(data, 10, cv_svm, re.compile(svm_path_expr), col_set=col_combos[0])
        epi_only = train_eval_model(data, 10, cv_svm, re.compile(svm_path_expr), col_set=col_combos[1])
        
        svm_results.loc[(coverage, seed, slice(None)), "struct"] = [r[0] for r in full_res]
        svm_results.loc[(coverage, seed, slice(None)), "epicurve"] = [r[0] for r in epi_only]

## Time Results

In [ ]:
time_dir = "/project/biocomplexity/cascade_analytics/2023-KDD/aggregates"

t_50_file = "exp4.T50.cell-1-3-5-7_outdegree-bins-1-2-10_boundary-outdegree-bins-1-5-10_epicurve-bin-1-5-10.csv"
data_50 = pd.read_csv(time_dir + t_50_file)


t_70_file = "exp4.T70.cell-1-3-5-7_outdegree-bins-1-2-10_boundary-outdegree-bins-1-5-10_epicurve-bin-1-5-10.csv"
data_70 = pd.read_csv(time_dir + t_70_file)

t_90_file = "exp4.T90.cell-1-3-5-7_outdegree-bins-1-2-10_boundary-outdegree-bins-1-5-10_epicurve-bin-1-5-10.csv"
data_90 = pd.read_csv(time_dir + t_90_file)

In [ ]:
lr_results = []
rf_results = []
svm_results = []

for data in [data_50, data_70, data_90]:
    for combo in col_combos:
        rf_results.append(train_eval_model(data, 10, cv_rf, re.compile(rf_path_expr),col_set=combo, n_iter=2))
        svm_results.append(train_eval_model(data, 10, cv_svm, re.compile(svm_path_expr),col_set=combo, n_iter=2))
        lr_results.append(train_eval_model(data, 10, gr_lr, re.compile(lr_path_expr), col_set=col, n_iter=2))


In [ ]:
# find max model over all training runs, this is used for SHAP analysis

model_dict = {} # map lr -> time -> model
for model_name, model_result in zip(["LR", "RF", "SVM"], [lr_results, rf_results, svm_results]):
    i = 0
    model_dict[model_name] = {}
    
    for t in ["50", "70", "90"]:
        for col in ["Epicurve+Structure", "Epicurve Only"]:
            if col != "Epicurve Only":
                result = model_result[i].copy()
                result.sort(key = lambda x: x[0])
                
                model_dict[model_name][t] = result[-1][1]
            i+=1

## Feature Significance

In [ ]:
import shap
import altair as alt

shap.initjs()

# To compute SHAP, we need feature subsets that were used to train each model
# 

def get_data(data, path_expr):
    
    inf_cols = data.columns[data.columns.map(lambda x: is_bucket(x, num_expr, 10))]
    path_cols = data.filter(regex=path_expr).columns
    # get all other columns
    out_degree_expr = re.compile("^out_degree_(\d+)_to_(\d+)")
    path_len_cols = data.filter(regex="^path_len_*").columns
    out_degree_cols = data.columns[data.columns.map(lambda x: is_bucket(x, out_degree_expr, 2))]

    feature_data = data[inf_cols.union(path_cols).union(path_len_cols).union(out_degree_cols)].copy()

    norm_cols = inf_cols.union(out_degree_cols).union(path_len_cols)

    feature_data[norm_cols] = feature_data[norm_cols].divide(data[inf_cols].sum(axis=1).values, axis=0)
    
    return feature_data

In [ ]:
# take SHAP measures for each row/column and average over the absolute value by feature-group

def arr_shap_df(raw_shap, path_expr):
    # return a dataframe mapping groups of columns to 
    # avg of abs. cols
    labeled_edge_idx = ~raw_shap.filter(regex=path_expr).droplevel(0, axis=1).columns.str.contains("-")
    labeled_edge_cols = raw_shap.filter(regex=path_expr).droplevel(0, axis=1).columns[labeled_edge_idx]

    labeled_path_cols = raw_shap.filter(regex=path_expr).droplevel(0, axis=1).columns[~labeled_edge_idx]
    path_len_cols = raw_shap.droplevel(0, axis=1).filter(regex="^path_len_*").columns

    out_degree_cols = raw_shap.droplevel(0, axis=1).columns[raw_shap.columns.map(lambda x: is_bucket(x[1], out_degree_expr, 2))]
    epicurve_cols = raw_shap.droplevel(0, axis=1).columns[raw_shap.columns.map(lambda x: is_bucket(x[1], num_expr, 10))]

    plot_data = {}
    
    for group_name, col_set in zip(["Labeled Edges", "Labeled paths", 
                                    "Path length", "Outdegree", "Epicurve"],
                    [labeled_edge_cols, labeled_path_cols, 
                    path_len_cols, out_degree_cols, epicurve_cols]):
        col_data = raw_shap.loc[:,(slice(None), col_set)].abs().mean(axis=1)
        plot_data[group_name] = col_data
        
    return pd.DataFrame(plot_data)

# create a plot of the spread by time
def plot_feature_time(shap_df, path_expr, feature_group, color):
    
    """
    return altair chart showing change in mean abs value shap over time for feature_group
    """
    time_dfs = []
    for time in shap_df.index.get_level_values("time").unique():
        time_df = arr_shap_df(shap_df.loc[time, :], path_expr)
        time_df["time"] = time
        time_dfs.append(time_df)
    chart_data = pd.concat(time_dfs)[["time", feature_group]]
    chart = alt.Chart(chart_data).mark_boxplot(color=color).encode(
        x = alt.X("time", scale=alt.Scale(domain=(40,100))),
        y = alt.Y(feature_group, scale=alt.Scale(domain=(0,0.2)), title="|Mean Shapley|"),
    ).properties(
        title=feature_group
    )
    
    return chart

In [ ]:
# Actually get the data

lr_data_50 = get_data(data_50, lr_path_expr)
lr_data_70 = get_data(data_70, lr_path_expr)
lr_data_90 = get_data(data_90, lr_path_expr)

rf_data_50 = get_data(data_50, rf_path_expr)
rf_data_70 = get_data(data_70, rf_path_expr)
rf_data_90 = get_data(data_90, rf_path_expr)

svm_data_50 = get_data(data_50, svm_path_expr)
svm_data_70 = get_data(data_70, svm_path_expr)
svm_data_90 = get_data(data_90, svm_path_expr)

In [ ]:
# LR SHAP computation

all_cols = lr_data_50.columns.union(lr_data_70.columns).union(lr_data_90.columns)

lr_shap_df = pd.DataFrame(
    index = pd.MultiIndex.from_product([[50, 70, 90],
                                      svm_data_50.index], names=["time", "cascade"]),
    columns = pd.MultiIndex.from_product([
        model_dict["LR"]["50"].classes_,
        all_cols],names=["class", "feature"])
)


for (time, model),data in zip(model_dict["LR"].items(), [lr_data_50, lr_data_70, lr_data_90]):
    
    explainer = shap.Explainer(model.predict_proba, data)
    shap_values = explainer(data)
    
    for i,class_val in enumerate(model.classes_):
        lr_shap_df.loc[(int(time), slice(None)), (class_val, data.columns)] = shap_values[:,:,i].values

In [ ]:
# RF SHAP computation

all_cols = rf_data_50.columns.union(rf_data_70.columns).union(rf_data_90.columns)
rf_shap_df = pd.DataFrame(
    index = pd.MultiIndex.from_product([[50, 70, 90],
                                      rf_data_50.index], names=["time", "cascade"]),
    columns = pd.MultiIndex.from_product([
        model_dict["RF"]["50"].classes_,
        all_cols],names=["class", "feature"])
)
# generate shap values for each model and time
# put into df of time x model x cell x feature

# rf

for (time, model),data in zip(model_dict["RF"].items(), [rf_data_50, rf_data_70, rf_data_90]):
    
    explainer = shap.Explainer(model.best_estimator_)
    shap_values = explainer(data)
    
    for i,class_val in enumerate(model.classes_):
        rf_shap_df.loc[(int(time), slice(None)), (class_val, data.columns)] = shap_values[:,:,i].values
        


In [ ]:
# SVM SHAP computation

all_cols = svm_data_50.columns.union(svm_data_70.columns).union(svm_data_90.columns)

svm_shap_df = pd.DataFrame(
    index = pd.MultiIndex.from_product([[50, 70, 90],
                                      svm_data_50.index], names=["time", "cascade"]),
    columns = pd.MultiIndex.from_product([
        model_dict["SVM"]["50"].classes_,
        all_cols],names=["class", "feature"])
)


for (time, model),data in zip(model_dict["SVM"].items(), [svm_data_50, svm_data_70, svm_data_90]):
    
    explainer = shap.Explainer(model.predict_proba, data)
    shap_values = explainer(data)
    
    for i,class_val in enumerate(model.classes_):
        svm_shap_df.loc[(int(time), slice(None)), (class_val, data.columns)] = shap_values[:,:,i].values

In [ ]:
color_map = {
    "Labeled Edges" : "#1b9e77",
    "Labeled paths" : "#d95f02",
    "Path length" : "#7570b3",
    "Outdegree" : "#e7298a",
    "Epicurve" : "#66a61e"
}

lr_chart = alt.hconcat().properties(title="LR")
rf_chart = alt.hconcat().properties(title="RF")
svm_chart = alt.hconcat().properties(title="SVM")

for feature_group in color_map:
    lr_chart |= plot_feature_time(lr_shap_df, lr_path_expr, feature_group, color_map[feature_group])
    rf_chart |= plot_feature_time(rf_shap_df, rf_path_expr, feature_group, color_map[feature_group])
    svm_chart |= plot_feature_time(svm_shap_df, svm_path_expr, feature_group, color_map[feature_group])
    
    
lr_chart
rf_chart
svm_chart

lr_chart & rf_chart & svm_chart